In [125]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
from re import sub 

base_link = "https://www.themoviedb.org/movie"

def get_page_content(url):
    get_headers = {'User-Agent': "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}
    response_page = requests.get(url, headers = get_headers )
    if not response_page.ok:
        raise Exception ("Failed to request. Status Code :- {}".format(response_page.status_code))
    else:
        page_content = response_page.text
        doc_page = BeautifulSoup(page_content, "html.parser")
        return doc_page
    
def movie_data_dict():
    scraped_dict = {  
                    'Title': [],
                    'User_rating': [], 
                    'Release_date':[], 
                    #'Director': [],
                    'Tagline': [],
                    'Runtime': [],
                    'Genre': [],
                    'Certification': [],
                    'Cast': [],
                    # 'Budget': [],
                    # 'Revenue': []
                    
                    }
    return scraped_dict

def user_rating(tag_user_score, i, scraped_dict):
    if tag_user_score[i]['data-percent'] == '0':
        scraped_dict['User_rating'].append('Not rated yet')
    else:
        scraped_dict['User_rating'].append(tag_user_score[i]['data-percent'])
        
def general_show_info(doc_page):
    base_link_1 = "https://www.themoviedb.org"
    tag_title = tag_premired_date = tag_shows_page = doc_page.find_all('div', {'class': 'card style_1'}) #it saves all of the card style_1 
    tag_user_score = doc_page.find_all('div', {"user_score_chart"}) 
    
    doc_2_list = []
    for link in tag_shows_page:
        doc_2_list.append(get_page_content("https://www.themoviedb.org" + link.h2.a['href']))
    return tag_title, tag_user_score, doc_2_list
        
def movie_genres(doc2_page, i):
    genres_tags = doc2_page[i].find('span', {"class": "genres"}).find_all('a')
    check_genre =[]
    
    for tag in genres_tags:
        check_genre.append(tag.text)
    return check_genre
    
def tagline_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('h3',{"class": 'tagline'}):
        scraped_dict['Tagline'].append(doc_2_list[i].find('h3',{"class": 'tagline'}).text)
    else:
        scraped_dict['Tagline'].append("No Tagline")

def remove(string):
    '''
    Removes everything inside [], a whitespace before that and *'s.
    '''
    if isinstance(string, str):
        string = sub(r'\s*\[.*\]\**|\n|,', '', string)
    return string

def runtime_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('span',{"class": 'runtime'}):
        scraped_dict['Runtime'].append(remove(doc_2_list[i].find('span',{"class": 'runtime'}).text))
    else:
        scraped_dict['Runtime'].append("No Runtime")
        
'''
def director_info(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('a',{"herf"}):
        scraped_dict['Director'].append(doc_2_list[i].find('a',{"herf": "/person/35453-pierre-morel"}).text)
    else:
        scraped_dict['Director'].append("No Director")
'''
        
def movie_cast(doc2_page, i):
    cast_tags = doc2_page[i].find_all('li', {'class': 'card'})
    cast_lis = []
    
    for t in cast_tags:
         cast_lis.append(t.p.text)
    
    return cast_lis

def certification(doc_2_list, i, scraped_dict):
    if doc_2_list[i].find('span',{"class": 'certification'}):
        scraped_dict['Certification'].append(remove(doc_2_list[i].find('span',{"class": 'certification'}).text))
    else:
        scraped_dict['Certification'].append("No Certification")

def scrape_budget(docs):
    
    slots = docs.find(attrs={"class" : "column no_bottom_pad"})    
    budget_string = slots.find_all("p")[-2].text
    
    pattern = r'\d+'
    
    budget = re.findall(pattern, budget_string)

    if len(budget)!=0:        
        string_budget = "".join(budget)
        return int(string_budget)/100
    else:
        return "NA"

def scrape_revenue(docs):
    slots = docs.find(attrs={"class" : "column no_bottom_pad"})
    revenue_string = slots.find_all("p")[-1].text
     
    pattern = r'\d+'

    revenue = re.findall(pattern, revenue_string)

    if len(revenue)!=0:
        string_revenue = "".join(revenue)
        return int(string_revenue)/100
        
    else :
        return "NA"


def create_budget_revenue_df(i, base_link):

    next_url = base_link + '?page={}'.format(i)
    doc_top = get_page_content(next_url)
    _, _, doc_2_lis = general_show_info(doc_top)

    # return doc_2_lis
    
    budget_revenue_dic = {"Budget":[],
                          "Revenue":[]}
    
    for i in range(0, len(doc_2_lis)):
        budget_revenue_dic["Budget"].append(scrape_budget(doc_2_lis[i]))
        budget_revenue_dic["Revenue"].append(scrape_revenue(doc_2_lis[i]))
        
    return pd.DataFrame(budget_revenue_dic)



def create_budget_revenue_csv(i, base_link):
    try:
        df = create_budget_revenue_df(i, base_link)
        df.to_csv(f"shows-data/revenue_budget_page_{i}.csv", index=None)
        print(f"CSV file for page {i} created successfully.")
    except Exception as e:
        print(f"An error occurred while creating CSV for page {i}: {e}")




def final_movie_details(t_title, t_user_score, docs_2_list):
    scraped_dict = movie_data_dict()
    for i in range (0, len(t_title)):
        scraped_dict['Title'].append(t_title[i].h2.text)
        user_rating(t_user_score, i, scraped_dict)   
        scraped_dict['Release_date'].append(t_title[i].p.text)
        tagline_info(docs_2_list, i, scraped_dict)  
        runtime_info(docs_2_list, i, scraped_dict) 
        certification(docs_2_list, i, scraped_dict)
        #director_info(docs_2_list, i, scraped_dict)
        scraped_dict['Genre'].append(movie_genres(docs_2_list, i))        
        scraped_dict['Cast'].append(movie_cast(docs_2_list, i))
        
    return pd.DataFrame(scraped_dict)


def create_page_df( i, dataframe_list):
    os.makedirs('shows-data', exist_ok = True)
    next_url = base_link + '?page={}'.format(i)
    doc_top = get_page_content(next_url)
    name_tag, viewer_score_tag, doc_2_lis = general_show_info(doc_top)
    print('scraping page {} :- {}'.format(i, next_url))
    dataframe_data = final_movie_details(name_tag, viewer_score_tag, doc_2_lis)
    dataframe_data.to_csv("shows-data/shows-page-{}.csv".format(i) , index = None)
    print(" ---> a CSV file has been created".format(i))
    dataframe_list.append(dataframe_data)
    

def scrap_final(base_link):
    dataframe_list = []
    for i in range(1, 11):
        create_page_df(i, dataframe_list)
        create_budget_revenue_csv(i, base_link)  # Call for each page

    total_dataframe = pd.concat(dataframe_list, ignore_index=True)
    csv_complete = total_dataframe.to_csv('shows-data/Total-dataframe.csv', index=None)
    print("\nA CSV file has been created")

    # Combine CSV files for budget and revenue
    combined_budget_revenue = pd.DataFrame()
    for i in range(1, 11):  # Adjust the range based on the number of pages
        budget_revenue = pd.read_csv(f'shows-data/revenue_budget_page_{i}.csv')
        combined_budget_revenue = pd.concat([combined_budget_revenue, budget_revenue], ignore_index=True)


    final_df = pd.concat([total_dataframe, combined_budget_revenue], axis=1)
    result_df = final_df[(~final_df["Budget"].isna()) & (~final_df["Revenue"].isna())]

    return result_df


In [126]:
result_df = scrap_final(base_link)
print(result_df)

scraping page 1 :- https://www.themoviedb.org/movie?page=1
 ---> a CSV file has been created
CSV file for page 1 created successfully.
scraping page 2 :- https://www.themoviedb.org/movie?page=2
 ---> a CSV file has been created
CSV file for page 2 created successfully.
scraping page 3 :- https://www.themoviedb.org/movie?page=3
 ---> a CSV file has been created
CSV file for page 3 created successfully.
scraping page 4 :- https://www.themoviedb.org/movie?page=4
 ---> a CSV file has been created
CSV file for page 4 created successfully.
scraping page 5 :- https://www.themoviedb.org/movie?page=5
 ---> a CSV file has been created
CSV file for page 5 created successfully.
scraping page 6 :- https://www.themoviedb.org/movie?page=6
 ---> a CSV file has been created
CSV file for page 6 created successfully.
scraping page 7 :- https://www.themoviedb.org/movie?page=7
 ---> a CSV file has been created
CSV file for page 7 created successfully.
scraping page 8 :- https://www.themoviedb.org/movie?pag